<a href="https://colab.research.google.com/github/arumshin-dev/python_conda_jupyter/blob/main/codeit/3_5_4_%E1%84%92%E1%85%A1%E1%86%AB%E1%84%80%E1%85%AE%E1%86%A8%E1%84%8B%E1%85%A5_FAQ_%E1%84%8E%E1%85%A2%E1%86%BA%E1%84%87%E1%85%A9%E1%86%BA_LangSmith.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# LangChain & LangSmith 실습: 한국어 FAQ 챗봇 만들기

## 학습 목표
1. LangChain의 핵심 컴포넌트 이해 및 활용
2. RAG(Retrieval Augmented Generation) 시스템 구축
3. LangSmith를 통한 모니터링 및 디버깅: 로그인후에 API 키 생성
    - https://smith.langchain.com/
4. 실전 FAQ 챗봇 개발

## 시나리오
IT 회사 FAQ 챗봇: 직원들의 자주 묻는 질문에 자동으로 답변하는 시스템

## Step 1: 환경 설정

In [ ]:
!pip install langchain langchain-openai langchain-community langsmith
!pip install chromadb tiktoken
!pip install python-dotenv

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.3/84.3 kB 3.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 37.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 50.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.7/64.7 kB 5.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 3.2 MB/s eta 0:00:00
  Attempting uninstall: requests
    Found existing installation: requests 2.32.4
    Uninstalling requests-2.32.4:
      Successfully uninstalled requests-2.32.4
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-colab 1.0.0 requires requests==2.32.4, but you have requests 2.32.5 which is incompatible.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 2.6 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Prepar

In [ ]:
import os
from getpass import getpass

# API 키 설정
os.environ["OPENAI_API_KEY"] = getpass("OpenAI API Key: ")
os.environ["LANGCHAIN_TRACING_V2"] = "true"
os.environ["LANGCHAIN_API_KEY"] = getpass("LangSmith API Key: ")
os.environ["LANGCHAIN_PROJECT"] = "korean-faq-chatbot-2"            # 프로젝트 이름

OpenAI API Key: ··········
LangSmith API Key: ··········


## Step 2: FAQ 데이터 준비

실제 회사에서 사용할 법한 HR FAQ 데이터를 준비합니다.

In [ ]:
# FAQ 데이터 (실제로는 파일이나 DB에서 가져옴)
faq_data = [
    {
        "질문": "연차는 어떻게 신청하나요?",
        "답변": "연차 신청은 사내 그룹웨어의 '전자결재' 메뉴에서 '휴가신청서'를 작성하면 됩니다. 최소 3일 전에 신청해야 하며, 팀장의 승인이 필요합니다. 긴급한 경우 전화로 먼저 알리고 사후 처리도 가능합니다."
    },
    {
        "질문": "월급날은 언제인가요?",
        "답변": "급여는 매월 25일에 지급됩니다. 25일이 주말이나 공휴일인 경우, 그 전 영업일에 지급됩니다. 급여명세서는 급여일 오전에 이메일로 발송됩니다."
    },
    {
        "질문": "재택근무 신청 방법을 알려주세요",
        "답변": "재택근무는 주 2회까지 가능합니다. 전날 오후 6시까지 팀장에게 슬랙으로 신청하면 됩니다. 재택근무 중에도 오전 10시-오후 5시는 업무 가능 상태여야 하며, 화상회의 참석이 가능해야 합니다."
    },
    {
        "질문": "건강검진은 언제 받나요?",
        "답변": "연간 건강검진은 입사 기념월에 받을 수 있습니다. 회사가 지정한 병원 리스트 중에서 선택 가능하며, 비용은 회사에서 전액 지원합니다. 인사팀으로 연락하시면 예약을 도와드립니다."
    },
    {
        "질문": "야근 수당은 어떻게 되나요?",
        "답변": "평일 오후 7시 이후 근무에 대해서는 야근 수당이 지급됩니다. 그룹웨어에서 야근 신청을 하고 팀장 승인을 받아야 합니다. 야근 수당은 시간당 기본급의 1.5배로 계산되며, 다음 달 급여에 포함됩니다."
    },
    {
        "질문": "경조사 휴가 기준은?",
        "답변": "경조사 휴가는 다음과 같이 제공됩니다. 본인 결혼: 5일, 자녀 결혼: 1일, 부모/배우자 부모 사망: 5일, 조부모/형제자매 사망: 3일. 경조사 발생 시 인사팀에 경조사 증빙서류를 제출해주세요."
    },
    {
        "질문": "점심시간은 몇 시부터 몇 시까지인가요?",
        "답변": "점심시간은 오후 12시부터 1시까지 1시간입니다. 팀별로 상황에 따라 11시 30분~1시 30분 사이에 탄력적으로 운영 가능합니다. 점심시간에는 자유롭게 외출 가능합니다."
    },
    {
        "질문": "사내 교육 프로그램이 있나요?",
        "답변": "분기별로 직무 교육과 온라인 교육 플랫폼을 제공합니다. 외부 교육이 필요한 경우 연간 100만원 한도 내에서 지원받을 수 있습니다. 교육 신청은 그룹웨어의 '교육신청' 메뉴에서 가능합니다."
    },
    {
        "질문": "주차는 어떻게 하나요?",
        "답변": "지하 주차장 이용이 가능하며, 입사 시 차량번호를 등록해야 합니다. 주차 공간이 한정되어 있어 선착순으로 배정됩니다. 주차 등록은 총무팀으로 신청하시면 됩니다."
    },
    {
        "질문": "복지 포인트는 어떻게 사용하나요?",
        "답변": "매년 1월 1일 기준으로 연간 150만원의 복지 포인트가 지급됩니다. 제휴 쇼핑몰에서 도서, 영화, 여행, 건강 등 다양한 분야에 사용 가능합니다. 사용 방법은 인사팀에서 발송하는 복지 가이드를 참조하세요."
    }
]

print(f"총 {len(faq_data)}개의 FAQ 데이터가 준비되었습니다.")
print("\n예시:")
print(f"Q: {faq_data[0]['질문']}")
print(f"A: {faq_data[0]['답변'][:50]}...")

총 10개의 FAQ 데이터가 준비되었습니다.

예시:
Q: 연차는 어떻게 신청하나요?
A: 연차 신청은 사내 그룹웨어의 '전자결재' 메뉴에서 '휴가신청서'를 작성하면 됩니다. 최소 ...


## Step 3: 기본 LangChain 챗봇 (RAG 없이)

먼저 간단한 LLM 기반 챗봇을 만들어봅니다.

In [ ]:
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser

# LLM 초기화
llm = ChatOpenAI(
    model="gpt-4o-mini",
    temperature=0  # 일관된 답변을 위해 0으로 설정
)

# 프롬프트 템플릿
template = """당신은 친절한 HR 담당자입니다.
직원의 질문에 정확하고 친절하게 답변해주세요.

질문: {question}

답변:"""

prompt = ChatPromptTemplate.from_template(template)

# 체인 구성
basic_chain = prompt | llm | StrOutputParser()

# 테스트
question = "연차는 어떻게 신청하나요?"
response = basic_chain.invoke({"question": question})
print(f"질문: {question}")
print(f"\n답변: {response}")

질문: 연차는 어떻게 신청하나요?

답변: 안녕하세요! 연차 신청 방법에 대해 안내해 드리겠습니다.

1. **신청서 작성**: 연차를 사용하고자 하는 날짜를 포함하여 연차 신청서를 작성해 주세요. 회사의 연차 신청 양식이 있다면 해당 양식을 사용하시면 됩니다.

2. **상사 승인**: 작성한 신청서를 상사에게 제출하여 승인을 받아야 합니다. 상사와의 대화에서 연차 사용 이유를 간단히 설명해 주시면 좋습니다.

3. **HR 부서 제출**: 상사의 승인을 받은 후, 최종 신청서를 HR 부서에 제출해 주세요. 이때, 제출 방법(이메일, 직접 제출 등)은 회사의 규정에 따라 다를 수 있으니 확인해 주세요.

4. **확인**: HR 부서에서 신청서를 확인한 후, 연차 사용이 승인되면 관련 내용을 안내해 드릴 것입니다.

추가로 궁금한 점이 있으시면 언제든지 문의해 주세요! 감사합니다.


### 🤔 문제점
- LLM이 일반적인 지식으로만 답변
- 회사 특정 정책을 모름
- 잘못된 정보를 제공할 수 있음 (환각 현상)

→ **RAG를 사용하여 해결!**

## Step 4: RAG 기반 FAQ 챗봇 구축

### 4.1 벡터 DB 생성

In [ ]:
from langchain_openai import OpenAIEmbeddings
from langchain_community.vectorstores import Chroma
from langchain_core.documents import Document

# FAQ를 Document 형식으로 변환
documents = []
for faq in faq_data:
    doc = Document(
        page_content=f"질문: {faq['질문']}\n답변: {faq['답변']}",
        metadata={"source": "HR FAQ", "question": faq['질문']}
    )
    documents.append(doc)

print(f"총 {len(documents)}개의 문서가 생성되었습니다.")
print("\n예시:")
print(documents[0].page_content)

총 10개의 문서가 생성되었습니다.

예시:
질문: 연차는 어떻게 신청하나요?
답변: 연차 신청은 사내 그룹웨어의 '전자결재' 메뉴에서 '휴가신청서'를 작성하면 됩니다. 최소 3일 전에 신청해야 하며, 팀장의 승인이 필요합니다. 긴급한 경우 전화로 먼저 알리고 사후 처리도 가능합니다.


In [ ]:
# 임베딩 모델 초기화
embeddings = OpenAIEmbeddings(model="text-embedding-3-small")

# 벡터 DB 생성
vectorstore = Chroma.from_documents(
    documents=documents,
    embedding=embeddings,
    collection_name="hr_faq"
)

print(" 벡터 DB가 생성되었습니다!")

 벡터 DB가 생성되었습니다!


### 4.2 검색 테스트

In [ ]:
# 유사도 검색 테스트
query = "휴가 신청은 어떻게 해요?"
results = vectorstore.similarity_search(query, k=3)

print(f"검색 쿼리: {query}\n")
print("검색 결과:")
for i, doc in enumerate(results, 1):
    print(f"\n[결과 {i}]")
    print(doc.page_content)
    print(f"메타데이터: {doc.metadata}")

검색 쿼리: 휴가 신청은 어떻게 해요?

검색 결과:

[결과 1]
질문: 연차는 어떻게 신청하나요?
답변: 연차 신청은 사내 그룹웨어의 '전자결재' 메뉴에서 '휴가신청서'를 작성하면 됩니다. 최소 3일 전에 신청해야 하며, 팀장의 승인이 필요합니다. 긴급한 경우 전화로 먼저 알리고 사후 처리도 가능합니다.
메타데이터: {'source': 'HR FAQ', 'question': '연차는 어떻게 신청하나요?'}

[결과 2]
질문: 경조사 휴가 기준은?
답변: 경조사 휴가는 다음과 같이 제공됩니다. 본인 결혼: 5일, 자녀 결혼: 1일, 부모/배우자 부모 사망: 5일, 조부모/형제자매 사망: 3일. 경조사 발생 시 인사팀에 경조사 증빙서류를 제출해주세요.
메타데이터: {'question': '경조사 휴가 기준은?', 'source': 'HR FAQ'}

[결과 3]
질문: 주차는 어떻게 하나요?
답변: 지하 주차장 이용이 가능하며, 입사 시 차량번호를 등록해야 합니다. 주차 공간이 한정되어 있어 선착순으로 배정됩니다. 주차 등록은 총무팀으로 신청하시면 됩니다.
메타데이터: {'question': '주차는 어떻게 하나요?', 'source': 'HR FAQ'}


### 4.3 RAG 체인 구성

In [ ]:
# 최신 LangChain 스타일(2025)
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnableParallel, RunnablePassthrough
from langchain_core.output_parsers import StrOutputParser
from langchain_openai import ChatOpenAI

retriever = vectorstore.as_retriever(search_kwargs={"k": 3})

# RAG 프롬프트 템플릿
rag_template = """당신은 회사의 HR 챗봇입니다.
아래 제공된 FAQ 정보를 바탕으로 직원의 질문에 정확하고 친절하게 답변해주세요.

FAQ 정보:
{context}

질문: {question}

답변 가이드:
1. FAQ에 정확한 답변이 있으면 그대로 전달하세요
2. FAQ에 없는 내용이면 "정확한 정보는 인사팀(내선 1234)으로 문의해주세요"라고 안내하세요
3. 친절하고 전문적인 톤을 유지하세요

답변:"""

# ChatPromptTemplate 사용
rag_prompt = ChatPromptTemplate.from_template(rag_template)

# 답변만 만드는 RAG 체인(LCEL)
answer_chain = (
    RunnableParallel(
        context=retriever,                # question을 retriever에 그대로 넣음
        question=RunnablePassthrough()    # question은 그대로 다음으로 전달
    )
    | rag_prompt
    | llm
    | StrOutputParser()
)

# "답변 + 출처 문서" 둘 다 반환하는 체인
qa_chain = RunnableParallel(
    answer=answer_chain,
    source_documents=retriever
)

print("=== RAG 체인이 생성되었습니다!")

=== RAG 체인이 생성되었습니다!


In [ ]:
# 테스트 실행
query = "연차에 대해 알고 싶어"
result = qa_chain.invoke(query)

print("--- 답변:")
print(result["answer"])
print("\n--- 사용된 문서 수:", len(result["source_documents"]))


--- 답변:
연차 신청은 사내 그룹웨어의 '전자결재' 메뉴에서 '휴가신청서'를 작성하시면 됩니다. 최소 3일 전에 신청해야 하며, 팀장의 승인이 필요합니다. 긴급한 경우에는 전화로 먼저 알리고 사후 처리도 가능합니다. 추가로 궁금한 점이 있으시면 언제든지 문의해 주세요!

--- 사용된 문서 수: 3


### 4.4 RAG 챗봇 테스트

In [ ]:
def ask_faq(question):
    """FAQ 챗봇에게 질문하기"""
    result = qa_chain.invoke(question)  # query dict 필요 없음

    answer = result["answer"]
    sources = result["source_documents"]

    print(f"💬 질문: {question}")
    print(f"\n🤖 답변: {answer}")
    print(f"\n📚 참고한 문서 수: {len(sources)}")

    if len(sources) > 0:
        print("\n관련 FAQ:")
        for i, doc in enumerate(sources, 1):
            # 문서의 메타데이터에 FAQ의 질문이 있다고 가정
            q = doc.metadata.get("question", "(질문 정보 없음)")
            print(f"  {i}. {q}")

    print("\n" + "="*80 + "\n")
    return result

In [ ]:
# 테스트 질문들
test_questions = [
    "연차는 어떻게 신청하나요?",
    "급여일이 언제에요?",
    "재택근무 가능한가요?",
    "회사에서 주식 투자 교육을 받을 수 있나요?"  # FAQ에 없는 질문
]

for q in test_questions:
    ask_faq(q)

💬 질문: 연차는 어떻게 신청하나요?

🤖 답변: 연차 신청은 사내 그룹웨어의 '전자결재' 메뉴에서 '휴가신청서'를 작성하시면 됩니다. 최소 3일 전에 신청해야 하며, 팀장의 승인이 필요합니다. 긴급한 경우에는 전화로 먼저 알리고 사후 처리도 가능합니다. 추가로 궁금한 점이 있으시면 언제든지 문의해 주세요!

📚 참고한 문서 수: 3

관련 FAQ:
  1. 연차는 어떻게 신청하나요?
  2. 주차는 어떻게 하나요?
  3. 재택근무 신청 방법을 알려주세요


💬 질문: 급여일이 언제에요?

🤖 답변: 급여는 매월 25일에 지급됩니다. 만약 25일이 주말이나 공휴일인 경우, 그 전 영업일에 지급됩니다. 급여명세서는 급여일 오전에 이메일로 발송됩니다. 추가로 궁금한 사항이 있으시면 언제든지 문의해 주세요!

📚 참고한 문서 수: 3

관련 FAQ:
  1. 월급날은 언제인가요?
  2. 야근 수당은 어떻게 되나요?
  3. 경조사 휴가 기준은?


💬 질문: 재택근무 가능한가요?

🤖 답변: 네, 재택근무는 주 2회까지 가능합니다. 전날 오후 6시까지 팀장에게 슬랙으로 신청하시면 됩니다. 재택근무 중에도 오전 10시부터 오후 5시까지는 업무 가능 상태여야 하며, 화상회의 참석이 가능해야 합니다. 추가로 궁금한 점이 있으시면 언제든지 문의해 주세요!

📚 참고한 문서 수: 3

관련 FAQ:
  1. 재택근무 신청 방법을 알려주세요
  2. 연차는 어떻게 신청하나요?
  3. 주차는 어떻게 하나요?


💬 질문: 회사에서 주식 투자 교육을 받을 수 있나요?

🤖 답변: 현재 제공된 FAQ 정보에는 주식 투자 교육에 대한 내용이 포함되어 있지 않습니다. 정확한 정보는 인사팀(내선 1234)으로 문의해 주시기 바랍니다. 도움이 필요하시면 언제든지 말씀해 주세요!

📚 참고한 문서 수: 3

관련 FAQ:
  1. 사내 교육 프로그램이 있나요?
  2. 건강검진은 언제 받나요?
  3. 복지 포인트는 어떻게 사용하나요?




## Step 5: LangSmith 모니터링

LangSmith에서 다음을 확인할 수 있습니다:
- 전체 체인 실행 과정
- 각 단계별 소요 시간
- 사용한 토큰 수와 비용
- 검색된 문서들
- LLM의 입출력

👉 https://smith.langchain.com 에서 'korean-faq-chatbot' 프로젝트를 확인하세요!

## Step 6: 대화 기록 기능 추가 (Memory)

In [ ]:
from operator import itemgetter

from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_core.runnables import RunnableWithMessageHistory
from langchain_core.output_parsers import StrOutputParser
from langchain_core.chat_history import InMemoryChatMessageHistory
from langchain_openai import ChatOpenAI

# 1) LLM & retriever 준비
llm = ChatOpenAI(model="gpt-4o-mini")  # 예시, 이미 있으시면 이건 생략
retriever = vectorstore.as_retriever(search_kwargs={"k": 3})

# 2) HR FAQ RAG 프롬프트 (대화 기록 + 컨텍스트 + 질문)
rag_template = """당신은 회사의 HR 챗봇입니다.
아래 제공된 FAQ 정보를 바탕으로 직원의 질문에 정확하고 친절하게 답변해주세요.

FAQ 정보:
{context}

질문: {question}

답변 가이드:
1. FAQ에 정확한 답변이 있으면 그대로 전달하세요
2. FAQ에 없는 내용이면 "정확한 정보는 인사팀(내선 1234)으로 문의해주세요"라고 안내하세요
3. 친절하고 전문적인 톤을 유지하세요

답변:"""

rag_prompt = ChatPromptTemplate.from_messages([
    (
        "system",
        "당신은 회사의 HR 챗봇입니다. "
        "FAQ와 이전 대화를 바탕으로 직원의 질문에 정확하고 친절하게 답변해주세요."
    ),
    MessagesPlaceholder("chat_history"),
    ("human", rag_template),
])

# 3) base_rag_chain: question + chat_history → context 검색 → LLM 호출

base_rag_chain = (
    {
        # question만 뽑아서 retriever에 넣어서 context 생성
        "context": itemgetter("question") | retriever,
        "question": itemgetter("question"),
        "chat_history": itemgetter("chat_history"),
    }
    | rag_prompt
    | llm
    | StrOutputParser()
)

# 4) 세션별 대화 히스토리 저장소
store = {}

def get_session_history(session_id: str):
    if session_id not in store:
        store[session_id] = InMemoryChatMessageHistory()
    return store[session_id]

# 5) 대화형 체인으로 래핑
conversational_chain = RunnableWithMessageHistory(
    base_rag_chain,
    get_session_history,
    input_messages_key="question",     # 유저 입력이 들어오는 필드
    history_messages_key="chat_history",  # 프롬프트에서 MessagesPlaceholder 이름
)

print("대화 기록 기능이 추가된 챗봇이 준비되었습니다!")

✅ 대화 기록 기능이 추가된 챗봇이 준비되었습니다!


In [ ]:
# 6) chat() 함수 정의
def chat(question, session_id="demo-user"):
    """대화형 HR FAQ 챗봇 질의 함수"""
    result = conversational_chain.invoke(
        {"question": question, "chat_history": []},  # chat_history는 RunnableWithMessageHistory가 내부 처리
        config={"configurable": {"session_id": session_id}},
    )
    print(f"💬 질문: {question}")
    print(f"🤖 답변: {result}\n")
    return result


# 7) 연속 대화 테스트
chat("연차는 어떻게 신청하나요?")
chat("최소 몇 일 전에 신청해야 하나요?")  # 이전 맥락 따라가야 함
chat("급여는 언제 받나요?")            # 새로운 주제


💬 질문: 연차는 어떻게 신청하나요?
🤖 답변: 연차 신청은 사내 그룹웨어의 '전자결재' 메뉴에서 '휴가신청서'를 작성하시면 됩니다. 최소 3일 전에 신청해야 하며, 팀장의 승인이 필요합니다. 긴급한 경우에는 전화로 먼저 알리고 사후 처리도 가능합니다. 추가로 궁금한 사항이 있다면 언제든지 물어보세요!

💬 질문: 최소 몇 일 전에 신청해야 하나요?
🤖 답변: 연차는 최소 3일 전에 신청해야 합니다. 추가로 궁금한 사항이 있으시면 언제든지 문의해 주세요!

💬 질문: 급여는 언제 받나요?
🤖 답변: 급여는 매월 25일에 지급됩니다. 만약 25일이 주말이나 공휴일인 경우, 그 전 영업일에 지급됩니다. 급여명세서는 급여일 오전에 이메일로 발송됩니다. 추가로 궁금한 사항이 있으시면 언제든지 문의해 주세요!



'급여는 매월 25일에 지급됩니다. 만약 25일이 주말이나 공휴일인 경우, 그 전 영업일에 지급됩니다. 급여명세서는 급여일 오전에 이메일로 발송됩니다. 추가로 궁금한 사항이 있으시면 언제든지 문의해 주세요!'

## Step 7: 성능 평가 및 개선

### 7.1 평가 데이터셋 만들기

In [ ]:
# 평가용 질문-답변 쌍
eval_data = [
    {
        "question": "연차 신청은 어떻게 하나요?",
        "expected_keywords": ["그룹웨어", "전자결재", "3일 전", "팀장 승인"]
    },
    {
        "question": "월급은 언제 들어오나요?",
        "expected_keywords": ["25일", "영업일", "급여명세서"]
    },
    {
        "question": "재택 근무가 가능한가요?",
        "expected_keywords": ["주 2회", "슬랙", "전날 오후 6시"]
    }
]

# 간단한 평가 함수
def evaluate_answer(question, answer, expected_keywords):
    score = sum(1 for keyword in expected_keywords if keyword in answer)
    total = len(expected_keywords)
    return score, total

In [ ]:
# 평가 실행
print(" 챗봇 성능 평가\n" + "="*50)
total_score = 0
total_keywords = 0

for eval_item in eval_data:
    # 예전: qa_chain.invoke({"query": eval_item["question"]})
    # 지금: qa_chain이 질문 문자열 하나를 받는다고 가정
    result = qa_chain.invoke(eval_item["question"])

    # 예전: result["result"]
    # 지금: result["answer"]
    answer = result["answer"]

    score, total = evaluate_answer(
        eval_item["question"],
        answer,
        eval_item["expected_keywords"]
    )

    total_score += score
    total_keywords += total

    print(f"\n질문: {eval_item['question']}")
    print(f"점수: {score}/{total} (포함된 키워드 비율)")
    print(f"답변: {answer[:100]}...")

accuracy = (total_score / total_keywords) * 100
print("\n" + "="*50)
print(f"전체 정확도: {accuracy:.1f}%")

 챗봇 성능 평가

질문: 연차 신청은 어떻게 하나요?
점수: 3/4 (포함된 키워드 비율)
답변: 연차 신청은 사내 그룹웨어의 '전자결재' 메뉴에서 '휴가신청서'를 작성하시면 됩니다. 최소 3일 전에 신청해야 하며, 팀장의 승인이 필요합니다. 긴급한 경우에는 전화로 먼저 알리고...

질문: 월급은 언제 들어오나요?
점수: 3/3 (포함된 키워드 비율)
답변: 급여는 매월 25일에 지급됩니다. 만약 25일이 주말이나 공휴일인 경우, 그 전 영업일에 지급됩니다. 급여명세서는 급여일 오전에 이메일로 발송됩니다. 추가적인 질문이 있으시면 언제...

질문: 재택 근무가 가능한가요?
점수: 3/3 (포함된 키워드 비율)
답변: 네, 재택근무는 주 2회까지 가능합니다. 전날 오후 6시까지 팀장에게 슬랙으로 신청하시면 됩니다. 재택근무 중에도 오전 10시부터 오후 5시까지는 업무 가능 상태여야 하며, 화상회...

전체 정확도: 90.0%


## Step 8: 실전 배포를 위한 추가 기능

In [ ]:
# Gradio를 사용한 간단한 UI: https://www.gradio.app/
!pip install gradio

In [ ]:
import gradio as gr

def chatbot_interface(message, history):
    """Gradio 챗봇 인터페이스"""
    # 질문 문자열 하나만 넣기
    result = qa_chain.invoke(message)

    answer = result["answer"]

    # 참고 문서에서 FAQ 질문 타이틀 뽑기
    sources = [
        doc.metadata.get("question", "N/A")
        for doc in result.get("source_documents", [])
    ]

    if sources:
        answer += "\n\n📚 참고한 FAQ:\n" + "\n".join(
            [f"- {s}" for s in sources[:2]]
        )

    return answer

demo = gr.ChatInterface(
    fn=chatbot_interface,
    title="🏢 HR FAQ 챗봇",
    description="회사 HR 관련 질문을 해보세요!",
    examples=[
        "연차는 어떻게 신청하나요?",
        "급여일이 언제인가요?",
        "재택근무가 가능한가요?",
        "건강검진은 언제 받나요?"
    ]
)

demo.launch(share=True)


/usr/local/lib/python3.12/dist-packages/gradio/chat_interface.py:347: UserWarning: The 'tuples' format for chatbot messages is deprecated and will be removed in a future version of Gradio. Please set type='messages' instead, which uses openai-style 'role' and 'content' keys.
  self.chatbot = Chatbot(


Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://3e98e5bb708d65b364.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


## 추가 학습 자료

1. **LangChain 공식 문서**: https://python.langchain.com/
2. **LangSmith 가이드**: https://docs.smith.langchain.com/
3. **RAG 심화**: https://python.langchain.com/docs/use_cases/question_answering/
4. **프롬프트 엔지니어링**: https://www.promptingguide.ai/
